### BELOW CODE IS TO WORK WITH SPARK IN COLAB, IGNORE IF NOT USING COLAB

In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [61.3 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/rest

In [2]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON']=sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']=sys.executable

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('week7').getOrCreate()
spark

In [39]:
path='/content/kddcup.data_10_percent_corrected'
data=spark.read.csv(path,inferSchema=True)
col_names = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent",
                "hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root",
                "num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login",
                "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
                "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
                "dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
                "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
data=data.toDF(*col_names)
data.show(5)
data.printSchema()

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_hos

### 1. Implement a PySpark script to handle any missing values and scale numerical features.

just data exploration

In [40]:
data.groupby('label').count().show()

+----------------+------+
|           label| count|
+----------------+------+
|    warezmaster.|    20|
|          smurf.|280790|
|            pod.|   264|
|           imap.|    12|
|           nmap.|   231|
|   guess_passwd.|    53|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|          satan.|  1589|
|           land.|    21|
|     loadmodule.|     9|
|      ftp_write.|     8|
|buffer_overflow.|    30|
|        rootkit.|    10|
|    warezclient.|  1020|
|       teardrop.|   979|
|           perl.|     3|
|            phf.|     4|
|       multihop.|     7|
|        neptune.|107201|
+----------------+------+
only showing top 20 rows



In [41]:
categorical=[]
numerical=[]
for col,coltype in data.dtypes:
  if coltype=='string':
    categorical.append(col)
  else:
    numerical.append(col)

In [42]:
numerical
categorical

['protocol_type', 'service', 'flag', 'label']

In [ ]:
# from pyspark.ml.feature import OneHotEncoder, StringIndexer
# from pyspark.ml import Pipeline
# indexer1 = StringIndexer().setInputCol('protocol_type').setOutputCol('protocol_type_indexed')
# encoder1 = OneHotEncoder().setInputCol('protocol_type_indexed').setOutputCol('protocol_type_vec')
# pipeline1=Pipeline(stages=[indexer1,encoder1])
# data=pipeline1.fit(data).transform(data)
# #-----------------------------------------------------
# indexer2 = StringIndexer(inputCol='service',outputCol='service_indexed')
# encoder2 = OneHotEncoder(inputCol='service_indexed',outputCol='service_vec')
# pipeline2=Pipeline(stages=[indexer2,encoder2])
# #--------------------------------------------------------
# indexer3= StringIndexer(inputCol='flag',outputCol='flag_indexed')
# encoder3= OneHotEncoder(inputCol='flag_indexed',outputCol='flag_vec')
# pipeline3=Pipeline(stages=[indexer3,encoder3])

In [ ]:
# pipeline1_model = pipeline1.fit(data)
# data = pipeline1_model.transform(data)

# pipeline2_model = pipeline2.fit(data)
# data = pipeline2_model.transform(data)

# pipeline3_model = pipeline3.fit(data)
# data = pipeline3_model.transform(data)

In [29]:
from pyspark.ml.feature import VectorAssembler
output='featureVector'
assembler=VectorAssembler(inputCols=numerical,outputCol=output)
data=assembler.transform(data)
data.select(["featureVector","label"]).show()

+--------------------+-------+
|       featureVector|  label|
+--------------------+-------+
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,6,8,19,2...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
|(38,[1,2,8,19,20,...|normal.|
+--------------------+-------+
only showing top 20 rows



In [30]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="featureVector",outputCol="scaledFeatures",withStd=True,withMean=True)
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)
data.select('featureVector','scaledFeatures').show()

+--------------------+--------------------+
|       featureVector|      scaledFeatures|
+--------------------+--------------------+
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,6,8,19,2...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.0677916519216...|
|(38,[1,2,8,19,20,...|[-0.067791

In [33]:
df=data.drop("protocol_type", "service", "flag")

### 2. Develop a PySpark script that uses the K-means algorithm to cluster data points.

In [57]:
from pyspark.ml.clustering import KMeans, KMeansModel
trainErr=[]
for k in list(range(60, 271, 30)):
  kmeans = KMeans().setK(k).setPredictionCol("cluster").setFeaturesCol("featureVector")
  model=kmeans.fit(df)
  trainErr.append(model.summary.trainingCost)

In [60]:
j=0
dictErr={}
for k in list(range(60, 271, 30)):
  print(k,trainErr[j])
  dictErr[trainErr[j]]=k
  j+=1

60 7751541631484.611
90 2388235091217.2354
120 1436504486348.055
150 1046375201472.1158
180 256531168364.16345
210 250114380913.06305
240 247015879019.6136
270 242244023270.63904


In [62]:
print(f"best k={dictErr[min(trainErr)]} with error={min(trainErr)}")

best k=270 with error=242244023270.63904


In [67]:
k = 270
# Define KMeans model
kmeans = KMeans().setK(k).setPredictionCol("cluster").setFeaturesCol("featureVector")
# Fit KMeans model to data
model = kmeans.fit(df)
# Transform data using the pipeline model and select relevant columns
transformed_data = model.transform(df).select("cluster", "label")
# Group by cluster and label, count occurrences, and order by cluster and label
count_by_cluster_label = transformed_data.groupBy("cluster", "label").count().orderBy("cluster", "label")
# Show the counts for the specified k value
count_by_cluster_label.show()

+-------+-------------+------+
|cluster|        label| count|
+-------+-------------+------+
|      0|   ftp_write.|     6|
|      0|guess_passwd.|    52|
|      0|        imap.|     9|
|      0|     ipsweep.|  1247|
|      0|        land.|    21|
|      0|    multihop.|     2|
|      0|     neptune.|107201|
|      0|        nmap.|   231|
|      0|      normal.| 41389|
|      0|         pod.|     5|
|      0|   portsweep.|   964|
|      0|     rootkit.|     6|
|      0|       satan.|  1588|
|      0|       smurf.|    24|
|      0|    teardrop.|   979|
|      0| warezclient.|   590|
|      0| warezmaster.|     1|
|      1|   portsweep.|     1|
|      2|      normal.|     1|
|      2| warezmaster.|    15|
+-------+-------------+------+
only showing top 20 rows

